In [ ]:
import time
import undetected_chromedriver as uc
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from curl_cffi import requests


def get_cookies():
    """
    Получает cookies и user-agent с сайта Ozon, используя маскированный браузер через undetected_chromedriver.

    Подробное описание процесса:

    1. Создаётся экземпляр браузера Chrome, замаскированный с помощью undetected_chromedriver.
       Это необходимо, чтобы обойти защиту от автоматизированных ботов (anti-bot системы),
       которые активно применяются на таких сайтах, как Ozon.
    
    2. WebDriverManager автоматически устанавливает подходящий ChromeDriver, 
       соответствующий установленной версии браузера Chrome.

    3. Включается режим неявного ожидания (60 секунд) — это означает, что WebDriver будет ждать 
       до 60 секунд появления нужных элементов на странице перед тем, как выбросить исключение. 
       Это важно при работе с динамически загружаемыми страницами.

    4. Загружается главная страница сайта https://www.ozon.ru.

    5. Ожидается появление элемента с селектором `#stickyHeader`. Этот элемент существует почти на всех страницах Ozon 
       и используется здесь как индикатор успешной загрузки и рендера основной части сайта.

    6. Через JavaScript (execute_script) извлекается значение `navigator.userAgent` — строка, описывающая тип браузера,
       операционную систему и другие параметры. Это нужно для точного копирования поведения браузера при последующих запросах.

    7. Из браузера извлекаются все cookies, установленные после загрузки страницы.
       Cookies могут включать информацию о сессии, уникальных идентификаторах, а также важные параметры для обхода авторизации.

    8. Список cookies преобразуется в словарь, где ключами являются имена cookies, а значениями — их значения.
       Такой формат нужен, чтобы их можно было передать в последующем HTTP-запросе с использованием curl_cffi.requests.

    Возвращает:
        Tuple[str, dict]: Кортеж, содержащий:
            - user_agent (str): строка User-Agent, полученная от браузера.
            - cookies_dict (dict): словарь cookies, необходимых для имитации браузера при последующем HTTP-запросе.
    """
    with uc.Chrome(service=ChromeService(ChromeDriverManager().install())) as driver:
        driver.implicitly_wait(60) # Ожидание элементы 60 сек.
        driver.get("https://www.ozon.ru")
        driver.find_element(By.CSS_SELECTOR, "#stickyHeader") # Поис Элемента 60 сек.
        user_agent = driver.execute_script("return navigator.userAgent")
        cookies = driver.get_cookies()

    cookies_dict = {i["name"]: i["value"] for i in cookies}
    return user_agent, cookies_dict


# Получаем user-agent и cookies с сайта Ozon с помощью функции выше.
user_agent, cookies_dict = get_cookies()


# Выполняем HTTP-запрос к странице поиска Ozon, используя полученные ранее cookies и user-agent.
response = requests.get(
    "https://www.ozon.ru/search/?text=ps+vita&from_global=true",
    cookies=cookies_dict,
    headers={"user-agent": user_agent}
)


# Сохраняем HTML-ответ от сервера (то, что вернул сайт) в файл на диск для последующего анализа или парсинга.
with open("html.html", "w", encoding="utf-8") as file:
    file.write(response.text)
